In [ ]:
!pip uninstall tensorflow
!pip uninstall keras

In [ ]:
!pip install tensorflow==2.12.0
!pip install keras

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
#from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# import csv data into dataframes
train = pd.read_csv(train_file_path, sep = '\t', names = ['label', 'body_text'], header = None)
train.dropna()
test = pd.read_csv(test_file_path, sep = '\t', names = ['label', 'body_text'], header = None)
test.dropna()

In [ ]:
train.tail()

In [ ]:
test.tail()

In [ ]:
#Remove stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stopword = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split()
                    if not word in stopword])
    txt = txt.lower()
    return txt

train['text_clean'] = train['body_text'].apply(lambda x: clean_txt(x))
test['text_clean'] = test['body_text'].apply(lambda x: clean_txt(x))

In [ ]:
train.tail()

In [ ]:
test.tail()

In [ ]:
train_message = train['text_clean']
train_label = np.array([0 if x=="ham" else 1 for x in train['label']])

test_message = test['text_clean']
test_label = np.array([0 if x=="ham" else 1 for x in test['label']])

In [ ]:
print(train['body_text'][0])
print(train_message[0])

In [ ]:
vocabulary_dict = {}
for text in train_message:
  for vocabulary in text:
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1

In [ ]:
VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message))

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

encoded_train_message = [one_hot(d, VOCAB_SIZE) for d in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
encoded_test_message = [one_hot(d, VOCAB_SIZE) for d in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence


tokens = Tokenizer(num_words=VOCAB_SIZE)
tokens.fit_on_texts(train_message)

# Transform each text to a sequence of integers
train_sequence = tokens.texts_to_sequences(train_message)
train_sequences_matrix = sequence.pad_sequences(train_sequence, maxlen=MAX_LENGTH)


test_sequence = tokens.texts_to_sequences(test_message)
test_sequences_matrix = sequence.pad_sequences(test_sequence, maxlen=max_len)

In [ ]:
print(sequences_matrix[:5])

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Embedding, Dense, Dropout, GlobalAveragePooling1D, LSTM, Input, Conv1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping

MODEL 1:

In [ ]:
model_extra = Sequential()
model_extra.add(Input(shape=(MAX_LENGTH,)))
model_extra.add(Embedding(VOCAB_SIZE, 100))
model_extra.add(Flatten())
model_extra.add(Dense(16, activation='relu'))
model_extra.add(Dropout(0.5))
model_extra.add(Dense(1, activation='sigmoid'))

model_extra.summary()

MODEL 2:

In [ ]:
model = tf.keras.Sequential()
model.add(Input(shape=(MAX_LENGTH,)))
model.add(Embedding(VOCAB_SIZE,50))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, verbose=1, mode='min', restore_best_weights=True)
model.fit(train_sequences_matrix, train_label, validation_data=(test_sequences_matrix, test_label), callbacks=[monitor], epochs=10, verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  class_dict = {
      0 : "ham",
      1 : "spam",
      }
  X = pd.Series(pred_text)
  text = X.apply(lambda x: clean_txt(x))
  seq = t.texts_to_sequences(text)
  sequence_matrix = sequence.pad_sequences(seq, maxlen=max_len)
  prediction = [model.predict(sequence_matrix)[0][0], class_dict[np.round(model.predict(sequence_matrix)[0][0])]]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    print(ans)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
